# Preparing the multispectral-image data and corresponding segmentation

### Export Sentinel-2 satellite imagery from Copernicus Scihub

Search for a city on https://scihub.copernicus.eu/dhus/#/home, filtering for Mission: Sentinel-2 and L1C product type. Export the data and unzip the folder locally. Detailed instructions are available at https://www.copernicus-user-uptake.eu/fileadmin/FPCUP/dateien/resources/2018-1-06/Guide_basics_satellite_data_english.pdf

### Export Urban Atlas land cover labels

Search for a city on https://land.copernicus.eu/local/urban-atlas/urban-atlas-2018?tab=download, download it and unzip the folder locally.

### Process

We create two GeoTiff files. First one contains 4 bands from a set of the satellite images, namely B02 (B), B03 (G), B04 (R), B08 (NIR). The second one contains the corresponding land use labels rasterized from Urban Atlas vector data. Steps that require manual change are marked in <span style="color:red">red</span>. We assume the dataset root file is at `datasets/final/`, change that accordingly.

#### <span style="color:red">Step 0</span>: Choose the filename stem of output files

In [1]:
city = "heidelberg"

<b><span style="color:red">Step 1</span>: Join the four bands of a single exported satellite image into an intermediate .vrt file</b>

In [2]:
%%bash

gdalbuildvrt \
-separate \
datasets/final/intermediate.vrt \
datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B02.jp2 \
datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B03.jp2 \
datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B04.jp2 \
datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B08.jp2

0...10...20...30...40...50...60...70...80...90...100 - done.


<b>Step 2: Look up information of the exported satellite image:</b>
- <b>projection saved under 'PROJCRS>ID'</b>
- <b>resolution saved under 'Pixel Size'</b>

In [3]:
%%bash

gdalinfo datasets/final/intermediate.vrt

Driver: VRT/Virtual Raster
Files: datasets/final/intermediate.vrt
       datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B02.jp2
       datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B03.jp2
       datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B04.jp2
       datasets/final/raw/S2/S2A_MSIL1C_20180912T103021_N0206_R108_T32UMV_20180912T143117.SAFE/GRANULE/L1C_T32UMV_A016836_20180912T103308/IMG_DATA/T32UMV_20180912T103021_B08.jp2
Size is 10980, 10980
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 32N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTH

<b><span style="color:red">Step 3</span>: Rasterize the Urban Atlas annotations.</b>

Special values that need to be adjusted:
* -l: the name of the layer where data is stored in .gpkg file. Commonly has the same name as the folder without the trailing version info.
* -tr: the resolution found in Step 2

In [4]:
%%bash -s "$city"

gdal_rasterize \
-a "code_2018" \
-l DE522L1_HEIDELBERG_UA2018 \
-of "GTiff" \
-a_nodata 0 \
-tr 10 10 \
-ot UInt16 \
datasets/final/raw/UA18/DE522L1_HEIDELBERG_UA2018_v013/Data/DE522L1_HEIDELBERG_UA2018_v013.gpkg \
datasets/final/annotations/"$1"_anno_orig.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


<b>Step 4: Look up the projection of rasterized annotations under 'PROJCRS>ID'</b>

In [5]:
%%bash -s "$city"

gdalinfo datasets/final/annotations/"$1"_anno_orig.tif

Driver: GTiff/GeoTIFF
Files: datasets/final/annotations/heidelberg_anno_orig.tif
Size is 4762, 5045
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        DATUM["European Terrestrial Reference System 1989",
            ELLIPSOID["GRS 1980",6378137,298.257222101,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4258]],
    CONVERSION["Europe Equal Area 2001",
        METHOD["Lambert Azimuthal Equal Area",
            ID["EPSG",9820]],
        PARAMETER["Latitude of natural origin",52,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",10,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["False easting",4321000,
            LENGTHUNIT["metre",1],
            ID["EPSG",8806]],
        PARAMETER["False northing",3210000,
        

<b><span style="color:red">Step 5</span>: Reproject rasterized annotations to the projection of the sattelite image</b>

Special values that need to be adjusted:
* -s_srs: projection of the annotations found in Step 4
* -t_srs: projection of the sattelite image found in Step 2
* -tr: the resolution found in Step 2

In [6]:
%%bash -s "$city"

gdalwarp \
-s_srs EPSG:3035 \
-t_srs EPSG:32632 \
-tr 10 10 \
datasets/final/annotations/"$1"_anno_orig.tif \
datasets/final/annotations/"$1"_anno.tif

Creating output file that is 4826P x 5108L.
Processing datasets/final/annotations/heidelberg_anno_orig.tif [1/1] : 0Using internal nodata values (e.g. 0) for image datasets/final/annotations/heidelberg_anno_orig.tif.
Copying nodata values from source datasets/final/annotations/heidelberg_anno_orig.tif to destination datasets/final/annotations/heidelberg_anno.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


<b>Step 6: Look up the bounding box of the annotations from "Upper Left" and "Lower Right"</b>

In [7]:
%%bash -s "$city"

gdalinfo \
datasets/final/annotations/"$1"_anno.tif

Driver: GTiff/GeoTIFF
Files: datasets/final/annotations/heidelberg_anno.tif
Size is 4826, 5108
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 32N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 32N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",9,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False easting",500000,
            LENGTHUNIT["metre",1],
            

<b><span style="color:red">Step 7</span>: Crop the satellite image to the annotated bounding box and save to file</b>

Special values that need to be adjusted:
* -te: the bounding box values extracted from Step 6 (\<Upper Left [0]\> \<Lower Right [1]\> \<Lower Right [0]\> \<Upper Left [1]\>)

In [8]:
%%bash -s "$city"

gdalwarp \
-te 459612.549 5446433.658 507872.549 5497513.658 \
datasets/final/intermediate.vrt \
datasets/final/intermediate_cropped.vrt

gdal_translate \
datasets/final/intermediate_cropped.vrt \
datasets/final/images/"$1"_s2.tif

Creating output file that is 4826P x 5108L.
Processing datasets/final/intermediate.vrt [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 4826, 5108
0...10...20...30...40...50...60...70...80...90...100 - done.


#### Step 8: Delete intermediate files

In [9]:
%%bash -s "$city"
rm -f datasets/final/annotations/"$1"_anno_orig.tif
rm -f datasets/final/intermediate.vrt
rm -f datasets/final/intermediate_cropped.vrt

### <span style="color:red">Optional: Multiple images</span>

Sometimes we need multiple satellite images for one city. In that case, make multiple Sentinel-2 exports and repeat Steps 8, 1, 7 (in this order) for each exported image (don't forget to change output filename in Step 7 to \_partX). After finishing, add all paths to the cell below and run it to obtain the final image.

In [17]:
%%bash -s "$city"

gdal_merge.py \
-o datasets/final/images/"$1"_s2.tif \
-n 0 \
-a_nodata 0 \
datasets/final/images/"$1"_part1.tif \
datasets/final/images/"$1"_part2.tif \
datasets/final/images/"$1"_part3.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


### Rearrange the annotations into final 5 classes 

For our use-case, we are only interested in the Level 1 CORINE Land Cover (CLC) classes: "Artificial areas", "Agricultural areas", "Forest and seminatural areas", "Wetlands", "Water bodies".

In [10]:
from osgeo import gdal
import numpy as np

IN_PATH = f"datasets/final/annotations/{city}_anno.tif"
OUT_PATH = f"datasets/final/annotations/{city}_anno.tif"

ds = gdal.Open(IN_PATH)
band = ds.GetRasterBand(1)
labels = band.ReadAsArray()
[rows, cols] = labels.shape

clc_level1 = {
    0: [0],
    1: [11100, 11210, 11220, 11230, 11240, 11300, 12100, 12210,
        12220, 12230, 12300, 12400, 13100, 13300, 13400, 14100, 14200],
    2: [21000, 22000, 23000, 24000, 25000],
    3: [31000, 32000, 33000],
    4: [40000],
    5: [50000]
}

for l1_label, codes in clc_level1.items():
    labels = np.where(np.isin(labels, codes), l1_label, labels)

driver = gdal.GetDriverByName("GTiff")
outdata = driver.Create(OUT_PATH, cols, rows, 1, gdal.GDT_UInt16)
outdata.SetGeoTransform(ds.GetGeoTransform())
outdata.SetProjection(ds.GetProjection())
outdata.GetRasterBand(1).WriteArray(labels)
outdata.GetRasterBand(1).SetNoDataValue(0)
outdata.FlushCache()

outdata = None
band = None
ds = None

### Optional: Extract segmentation preview

In [11]:
from PIL import Image
from osgeo import gdal
import numpy as np


sentinel_data = gdal.Open(f"datasets/final/images/{city}_s2.tif")
label_data = gdal.Open(f"datasets/final/annotations/{city}_anno.tif")

b = sentinel_data.GetRasterBand(1).ReadAsArray()
g = sentinel_data.GetRasterBand(2).ReadAsArray()
r = sentinel_data.GetRasterBand(3).ReadAsArray()

labels = label_data.GetRasterBand(1).ReadAsArray()
labels = labels.astype(np.uint8)

im = np.dstack([r,g,b])

maxval = 3558
im_truncated = np.where(im < maxval, im, maxval)

im_normalized = (im_truncated - im_truncated.min()) / (im_truncated.max() - im_truncated.min())
im_normalized = (im_normalized * 255).astype(np.uint8)

labels_rgb = np.dstack([labels, labels, labels])

labels_rgb[labels == 1] = [255,   0,   0]  # Artificial areas (RED)
labels_rgb[labels == 2] = [255, 255,   0]  # Agriculture areas (YELLOW)
labels_rgb[labels == 3] = [0  , 255,   0]  # Forest and semi-natural areas (GREEN)
labels_rgb[labels == 4] = [0  , 255, 255]  # Wetlands (CYAN)
labels_rgb[labels == 5] = [0  ,   0, 255]  # Water bodies (BLUE)

im_normalized_pil = Image.fromarray(im_normalized)
im_normalized_pil = im_normalized_pil.convert("RGBA")

labels_rgb_pil = Image.fromarray(labels_rgb)
labels_rgb_pil = labels_rgb_pil.convert("RGBA")

segmented_im = Image.blend(
    im_normalized_pil,
    labels_rgb_pil,
    0.2
)

segmented_im.save(f"datasets/final/preview/{city}.png", "PNG")